In [1]:
from IPython.display import display, Markdown
import polars as pl
import mistune

In [2]:
dataset_pl = pl.read_parquet("../codecontests_cot_sft_v2.parquet")

In [ ]:
dataset_pl

In [93]:
import pprint
import mistune.renderers
import mistune.renderers.markdown
from collections import defaultdict
from typing import Literal
from pprint import pprint

ModelChoice = Literal["gpt-4o", "deepseek"]

model_choice: ModelChoice = "gpt-4o"

markdown_renderer = mistune.create_markdown(renderer=None)

heading_freqs = defaultdict(int)

n_with_steps = 0


def recursive_get_all_children(element, children_list, disallowed_types=[]):
    if "type" in element and element["type"] in disallowed_types:
        return children_list
    if "children" in element:
        for child in element["children"]:
            recursive_get_all_children(child, children_list, disallowed_types)
    else:
        children_list.append(element)
    return children_list


cot_formatted_rows = []

for i in range(len(dataset_pl)):
    # display(Markdown('## Problem'))
    problem_str = dataset_pl[i]["problem"][0]
    problem_name = dataset_pl[i]["name"][0]
    try:
        solution_str = dataset_pl[i]["completions"][0][0]
    except:
        print("warn: no solution")
        continue
    problem_md = markdown_renderer(problem_str)
    solution_md = markdown_renderer(solution_str)
    # print(solution_str)
    # display(solution_md)
    thoughts = []
    solution_code = None
    for i, element in enumerate(solution_md):
        # Deepseek has a preamble, then the steps; 4o prints a paragraph then the steps, so the first
        # list is always the steps
        if model_choice == "deepseek":
            if element["type"] == "heading":
                heading_text = element["children"][0]["raw"].lower()
                has_steps = False
                required_phrases = ["reasoning", "steps", "approach"]

                if (
                    any(phrase in heading_text for phrase in required_phrases)
                    and "code" not in heading_text
                ):
                    has_steps = True
                if has_steps:
                    n_with_steps += has_steps
                    heading_freqs[heading_text] += 1
                    steps_list_idx = i + 1
                    if solution_md[i + 1]["type"] == "blank_line":
                        steps_list_idx = i + 2
                    steps_list_element = solution_md[steps_list_idx]
                    for step in steps_list_element["children"]:
                        # 0 is the prefix + :
                        if "children" not in step or len(step["children"]) < 2:
                            print(step)
                            continue
                        bullet_point_text = step["children"][1]
                        if "children" not in bullet_point_text:
                            print(bullet_point_text)
                            continue
                        for sub_step in bullet_point_text["children"]:
                            for sub_sub_step in sub_step["children"]:
                                all_text = recursive_get_all_children(sub_sub_step, [])
                                if any(["raw" not in x for x in all_text]):
                                    print(all_text)
                                    continue
                                all_text_str = "".join([t["raw"] for t in all_text])
                                thoughts.append(all_text_str)
        elif model_choice == "gpt-4o":
            if element["type"] == "list":
                # pprint(element)
                for child in element["children"]:
                    for sub_child in child["children"]:
                        all_text = recursive_get_all_children(sub_child, [], ["strong"])
                        all_text = [t["raw"] for t in all_text if "raw" in t]
                        all_text = [t.lstrip(": ") for t in all_text]
                        all_text = [t for t in all_text if len(t) > 0 and t != "\n"]
                        all_text_str = " ".join([t for t in all_text])
                        if len(all_text_str) > 0:
                            thoughts.append(all_text_str)
        if element["type"] == "block_code" and solution_code is None:
            if "raw" not in element:
                print(element)
                continue
            solution_code = element["raw"]

        if solution_code is not None and len(thoughts) > 0:
            cot_formatted_rows.append(
                {
                    "problem": problem_str,
                    "code": solution_code,
                    "thoughts": thoughts,
                    "problem_name": problem_name,
                }
            )
            break


In [95]:
cot_formatted_rows

[{'problem': "There are n persons who initially don't know each other. On each morning, two of them, who were not friends before, become friends.\n\nWe want to plan a trip for every evening of m days. On each trip, you have to select a group of people that will go on the trip. For every person, one of the following should hold: \n\n  * Either this person does not go on the trip, \n  * Or at least k of his friends also go on the trip. \n\n\n\nNote that the friendship is not transitive. That is, if a and b are friends and b and c are friends, it does not necessarily imply that a and c are friends.\n\nFor each day, find the maximum number of people that can go on the trip on that day.\n\nInput\n\nThe first line contains three integers n, m, and k (2 ≤ n ≤ 2 ⋅ 10^5, 1 ≤ m ≤ 2 ⋅ 10^5, 1 ≤ k < n) — the number of people, the number of days and the number of friends each person on the trip should have in the group.\n\nThe i-th (1 ≤ i ≤ m) of the next m lines contains two integers x and y (1≤ x

In [96]:
out_rows_pl = pl.DataFrame(cot_formatted_rows)
out_rows_pl.write_parquet("codecontests_cot_sft_formatted_thoughts_v2_gpt.parquet")

In [97]:
cot_formatted_rows

[{'problem': "There are n persons who initially don't know each other. On each morning, two of them, who were not friends before, become friends.\n\nWe want to plan a trip for every evening of m days. On each trip, you have to select a group of people that will go on the trip. For every person, one of the following should hold: \n\n  * Either this person does not go on the trip, \n  * Or at least k of his friends also go on the trip. \n\n\n\nNote that the friendship is not transitive. That is, if a and b are friends and b and c are friends, it does not necessarily imply that a and c are friends.\n\nFor each day, find the maximum number of people that can go on the trip on that day.\n\nInput\n\nThe first line contains three integers n, m, and k (2 ≤ n ≤ 2 ⋅ 10^5, 1 ≤ m ≤ 2 ⋅ 10^5, 1 ≤ k < n) — the number of people, the number of days and the number of friends each person on the trip should have in the group.\n\nThe i-th (1 ≤ i ≤ m) of the next m lines contains two integers x and y (1≤ x

In [98]:
from openai.types.chat.chat_completion_message_param import ChatCompletionMessageParam
from typing import Sequence


def format_out_row_str(row: dict) -> Sequence[ChatCompletionMessageParam]:
    problem, code, thoughts = row["problem"], row["code"], row["thoughts"]

    thoughts_list = [f"<thought>{t}</thought>" for t in thoughts]
    thoughts_str = "\n".join(thoughts_list)
    code_str = f"<solution>{code}</solution>"
    conv: Sequence[ChatCompletionMessageParam] = [
        {
            "role": "user",
            "content": f"Solve the following programming problem in Python.\n{problem}",
        },
        {
            "role": "assistant",
            "content": f"{thoughts_str}\n\n{code_str}",
        },
    ]
    return conv


display(pprint(format_out_row_str(cot_formatted_rows[0])))

[{'content': 'Solve the following programming problem in Python.\n'
             "There are n persons who initially don't know each other. On each "
             'morning, two of them, who were not friends before, become '
             'friends.\n'
             '\n'
             'We want to plan a trip for every evening of m days. On each '
             'trip, you have to select a group of people that will go on the '
             'trip. For every person, one of the following should hold: \n'
             '\n'
             '  * Either this person does not go on the trip, \n'
             '  * Or at least k of his friends also go on the trip. \n'
             '\n'
             '\n'
             '\n'
             'Note that the friendship is not transitive. That is, if a and b '
             'are friends and b and c are friends, it does not necessarily '
             'imply that a and c are friends.\n'
             '\n'
             'For each day, find the maximum number of people that c

None

In [101]:
from tqdm import tqdm
conv_out = []
for row in tqdm(cot_formatted_rows):
    conv_out.append({"conversation": format_out_row_str(row), "problem_name": row["problem_name"]})
out_rows_pl = pl.DataFrame(conv_out)

100%|██████████| 336/336 [00:00<00:00, 142496.07it/s]


In [102]:
display(out_rows_pl)
out_rows_pl.write_parquet("codecontests_cot_sft_formatted_thoughts_conversations.parquet")

shape: (336, 2)
┌─────────────────────────────────┬─────────────────────────────────┐
│ conversation                    ┆ problem_name                    │
│ ---                             ┆ ---                             │
│ list[struct[2]]                 ┆ str                             │
╞═════════════════════════════════╪═════════════════════════════════╡
│ [{"user","Solve the following … ┆ 1037_E. Trips                   │
│ [{"user","Solve the following … ┆ 1060_A. Phone Numbers           │
│ [{"user","Solve the following … ┆ 1101_A. Minimum Integer         │
│ [{"user","Solve the following … ┆ 1189_D1. Add on a Tree          │
│ [{"user","Solve the following … ┆ 1208_D. Restore Permutation     │
│ …                               ┆ …                               │
│ [{"user","Solve the following … ┆ 117_B. Very Interesting Game    │
│ [{"user","Solve the following … ┆ 1198_C. Matching vs Independen… │
│ [{"user","Solve the following … ┆ 1215_B. The Number of Products  │
│ [{"user","Solve the following … ┆ 1256_E. Yet Another Division I… │
│ [{"user","Solve the following … ┆ 127_E. E-reader Display         │
└─────────────────────────────────┴─────────────────────────────────┘

In [ ]:
sorted_freqs_table = sorted(heading_freqs.items(), key=lambda x: x[1], reverse=True)
sorted_freqs_table